## Import packages

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from helper import series_to_supervised
from tensorflow.keras.models import load_model
from preprocess import water_postprocess
from scipy.stats import mannwhitneyu, wilcoxon, ttest_rel
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from performance import metrics_s1_t1

2024-02-03 14:55:52.034830: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

## Preprocess data

In [4]:
dataset = pd.read_csv('../data/energy_weather.csv', index_col=0)
# https://www.kaggle.com/datasets/nicholasjhana/energy-consumption-generation-prices-and-weather

dataset.fillna(0, inplace=True)
data = dataset
data

,price,price_dayahead,gen_coal,gen_gas,load_actual,gen_lig,gen_oil,gen_oth_renew,pressure_Barcelona,pressure_Bilbao,...,wind_deg_Bilbao,clouds_all_Bilbao,gen_hyd_river,wind_deg_Seville,wind_speed_Barcelona,wind_speed_Valencia,wind_speed_Bilbao,gen_wind,wind_speed_Madrid,gen_hyd_pump
time,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00+00:00,64.92,48.10,4755.0,5196.0,24382.0,328.0,158.0,71.0,1035.0,1035.0,...,229.0,0.0,1009.0,21.0,7.0,1.0,0.0,5890.0,1.0,920.0
2015-01-01 01:00:00+00:00,64.48,47.33,4581.0,4857.0,22734.0,323.0,157.0,73.0,1036.0,1036.0,...,224.0,0.0,973.0,27.0,7.0,0.0,1.0,5461.0,1.0,1164.0
2015-01-01 02:00:00+00:00,59.32,42.27,4131.0,4314.0,21286.0,254.0,160.0,75.0,1036.0,1035.0,...,225.0,0.0,949.0,27.0,7.0,0.0,1.0,5238.0,1.0,1503.0
2015-01-01 03:00:00+00:00,56.04,38.41,3840.0,4130.0,20264.0,187.0,156.0,74.0,1036.0,1035.0,...,221.0,0.0,953.0,27.0,7.0,0.0,1.0,4935.0,1.0,1826.0
2015-01-01 04:00:00+00:00,53.63,35.72,3590.0,4038.0,19905.0,178.0,156.0,74.0,1037.0,1035.0,...,224.0,0.0,952.0,57.0,5.0,2.0,1.0,4618.0,0.0,2109.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 18:00:00+00:00,77.02,68.85,2628.0,7634.0,30653.0,0.0,178.0,95.0,1027.0,1033.0,...,57.0,0.0,1135.0,30.0,1.0,2.0,0.0,3113.0,1.0,1.0
2018-12-31 19:00:00+00:00,76.16,68.40,2566.0,7241.0,29735.0,0.0,174.0,95.0,1027.0,1034.0,...,0.0,0.0,1172.0,30.0,3.0,1.0,1.0,3288.0,1.0,1.0
2018-12-31 20:00:00+00:00,74.30,66.88,2422.0,7025.0,28071.0,0.0,168.0,94.0,1028.0,1034.0,...,140.0,0.0,1148.0,50.0,4.0,3.0,1.0,3503.0,1.0,50.0


In [5]:
values = data.values

# specify the number of lag hours
n_hours = 24*3
n_features = data.shape[-1]
k = 12
split1 = 0.7
split2 = 0.85

# frame as supervised learning
reframed = series_to_supervised(values, n_hours, k)
print("reframed.shape:", reframed.shape)

reframed.shape: (34980, 2184)


In [6]:
# split into train and test sets
reframed_values = reframed.values
n_train_hours = int(len(reframed_values)*split1)
n_valid_hours = int(len(reframed_values)*split2)

train = reframed_values[:n_train_hours, :]
val = reframed_values[n_train_hours:n_valid_hours, :]
test = reframed_values[n_valid_hours:, :]


# split into input and outputs
n_obs = n_hours * n_features
feature_idx = 4
train_X, train_y = train[:, :n_obs], train[:, [n_obs + feature_idx + n_features * i for i in range(k)]]
val_X, val_y = val[:, :n_obs], val[:, [n_obs + feature_idx + n_features * i for i in range(k)]]
test_X, test_y = test[:, :n_obs], test[:, [n_obs + feature_idx + n_features * i for i in range(k)]]


print("train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape", 
      train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape
     )

train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape (24486, 1872) (24486, 12) (5247, 1872) (5247, 12) (5247, 1872) (5247, 12)


In [7]:
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))

train_X = scaler.fit_transform(train_X)
train_y = scaler.fit_transform(train_y)

val_X = scaler.fit_transform(val_X)
val_y = scaler.fit_transform(val_y)

test_X = scaler.fit_transform(test_X)
test_y = scaler.fit_transform(test_y)

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
val_X = val_X.reshape((val_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))

print("train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape", 
      train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape
     )

train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape (24486, 72, 26) (24486, 12) (5247, 72, 26) (5247, 12) (5247, 72, 26) (5247, 12)


### Load threshold

In [8]:
train_X_pm = train_X[:, 0, feature_idx]
print(train_X_pm.shape)

val_X_pm = val_X[:, 0, feature_idx]
print(val_X_pm.shape)

test_X_pm = test_X[:, 0, feature_idx]
print(test_X_pm.shape)

(24486,)
(5247,)
(5247,)


In [9]:
percentile = 95

merged_array = np.concatenate((train_X_pm, val_X_pm, test_X_pm))

percentile_pm = np.percentile(merged_array, percentile)

print("{}th Percentile of Daily Rain:".format(percentile), percentile_pm)

95th Percentile of Daily Rain: 0.793594933402977


### train_X_filter

In [10]:
train_X_extreme = train_X[train_X_pm > percentile_pm]
print(train_X_extreme.shape)

train_y_extreme = train_y[train_X_pm > percentile_pm]
print(train_y_extreme.shape)

(976, 72, 26)
(976, 12)


In [11]:
train_X_normal = train_X[train_X_pm <= percentile_pm]
print(train_X_normal.shape)

train_y_normal = train_y[train_X_pm <= percentile_pm]
print(train_y_normal.shape)

(23510, 72, 26)
(23510, 12)


### val_X_filter

In [12]:
val_X_extreme = val_X[val_X_pm > percentile_pm]
print(val_X_extreme.shape)

val_y_extreme = val_y[val_X_pm > percentile_pm]
print(val_y_extreme.shape)

(501, 72, 26)
(501, 12)


In [13]:
val_X_normal = val_X[val_X_pm <= percentile_pm]
print(val_X_normal.shape)

val_y_normal = val_y[val_X_pm <= percentile_pm]
print(val_y_normal.shape)

(4746, 72, 26)
(4746, 12)


### test_X_filter

In [14]:
test_X_extreme = test_X[test_X_pm > percentile_pm]
print(test_X_extreme.shape)

test_y_extreme = test_y[test_X_pm > percentile_pm]
print(test_y_extreme.shape)

(272, 72, 26)
(272, 12)


In [15]:
test_X_normal = test_X[test_X_pm <= percentile_pm]
print(test_X_normal.shape)

test_y_normal = test_y[test_X_pm <= percentile_pm]
print(test_y_normal.shape)

(4975, 72, 26)
(4975, 12)


## Test model

In [18]:
# ws_threshold = 2.58
time_index = 0

#### Extreme

In [23]:
# saved_model = load_model('../saved_models_mlp/load_N.h5') 
# saved_model = load_model('../saved_models_mlp/load_E.h5')

# saved_model = load_model('../saved_models_cnn/load_all.h5') 
# saved_model = load_model('../saved_models_mlp/load_all_95_ft.h5')


# saved_model = load_model('../saved_models_mlp/load_all_weighted_IPF_95.h5')
# saved_model = load_model('../saved_models_mlp/load_all_weighted_IPF_95_ft.h5')


# saved_model = load_model('../saved_models_mlp/load_all_weighted_EVT_95.h5')
# saved_model = load_model('../saved_models_mlp/load_all_weighted_EVT_95_ft.h5')


saved_model = load_model('../saved_models_mlp/load_all_weighted_META_95.h5')
# saved_model = load_model('../saved_models_mlp/load_all_weighted_META_95_ft.h5')


yhat_extreme = saved_model.predict(test_X_extreme)


inv_yhat_extreme = scaler.inverse_transform(yhat_extreme)
inv_y_extreme = scaler.inverse_transform(test_y_extreme)
test_errors_extreme = inv_yhat_extreme - inv_y_extreme

# print('MAE = {}'.format(float("{:.6f}".format(mae(inv_y_extreme, inv_yhat_extreme)))))
# print('RMSE = {}'.format(float("{:.6f}".format(np.sqrt(mse(inv_y_extreme, inv_yhat_extreme))))))

In [24]:
metrics_s1_t1(inv_y_extreme.min(), time_index, inv_y_extreme, inv_yhat_extreme, test_errors_extreme)

Peformance when water level is over 19894.0 ft 

------ MAE & RMSE ------
MAE = 1877.710209
RMSE = 2298.483401 

------ Max Errors (t+1 at S1) ------
Max Error of Over Estimation: 6042.30078125
Max Error of Under Estimation: -2460.28515625
Max Abs Error of Under Estimation: 6042.30078125 

------ Time # (t+1 at S1) ------
Time# of Over Estimation: 230
Time# of Under Estimation: 42 

------ Area (t+1 at S1) ------
Area of Over Estimation: 432933.771484375
Area of Under Estimation: -34013.251953125015


#### Normal & Extreme hen water level is over threshold 2.58 feet (95 percentile)

In [25]:
# saved_model = load_model('../saved_models_mlp/load_all.h5') 
# saved_model = load_model('../saved_models_mlp/load_all_95_ft.h5')


# saved_model = load_model('../saved_models_mlp/load_all_weighted_IPF_95.h5')
# saved_model = load_model('../saved_models_mlp/load_all_weighted_IPF_95_ft.h5')


# saved_model = load_model('../saved_models_mlp/load_all_weighted_EVT_95.h5')
# saved_model = load_model('../saved_models_mlp/load_all_weighted_EVT_95_ft.h5')


saved_model = load_model('../saved_models_mlp/load_all_weighted_META_95.h5')
# saved_model = load_model('../saved_models_mlp/load_all_weighted_META_95_ft.h5')


yhat = saved_model.predict(test_X)


inv_yhat = scaler.inverse_transform(yhat)
inv_y = scaler.inverse_transform(test_y)
test_errors = inv_yhat - inv_y

metrics_s1_t1(inv_y.min(), time_index, inv_y, inv_yhat, test_errors)

Peformance when water level is over 18179.0 ft 

------ MAE & RMSE ------
MAE = 1493.515606
RMSE = 1951.430545 

------ Max Errors (t+1 at S1) ------
Max Error of Over Estimation: 5038.830078125004
Max Error of Under Estimation: -4428.228515625
Max Abs Error of Under Estimation: 5038.830078125004 

------ Time # (t+1 at S1) ------
Time# of Over Estimation: 4348
Time# of Under Estimation: 898 

------ Area (t+1 at S1) ------
Area of Over Estimation: 5472538.93359375
Area of Under Estimation: -518307.21484375


#### hyperparameter - frozen layers

In [23]:
layers = [1, 5, 9, 13, 17]

for layer in layers:
    print('layer: {}'.format(layer))    
    saved_model = load_model('../saved_models_hyper/load_all_weighted_META_95_ft_{}.h5'.format(layer))
    yhat_extreme = saved_model.predict(test_X_extreme)

    inv_yhat_extreme = scaler.inverse_transform(yhat_extreme)
    inv_y_extreme = scaler.inverse_transform(test_y_extreme)
    test_errors_extreme = inv_yhat_extreme - inv_y_extreme
    
    print('MAE = {}'.format(float("{:.6f}".format(mae(inv_y_extreme, inv_yhat_extreme)))))
    print('RMSE = {}'.format(float("{:.6f}".format(np.sqrt(mse(inv_y_extreme, inv_yhat_extreme))))))
    print('-------------------------')

layer: 1
MAE = 1151.865394
RMSE = 1464.077215
-------------------------
layer: 5
MAE = 1249.73877
RMSE = 1540.492848
-------------------------
layer: 9
MAE = 1240.913885
RMSE = 1534.453236
-------------------------
layer: 13
MAE = 1246.620784
RMSE = 1543.006424
-------------------------
layer: 17
MAE = 1399.695916
RMSE = 1694.042388
-------------------------


### p-values

In [20]:
saved_model = load_model('../saved_models_mlp/load_all.h5') 

yhat_extreme = saved_model.predict(test_X_extreme)
inv_yhat_extreme = scaler.inverse_transform(yhat_extreme)
inv_y_extreme = scaler.inverse_transform(test_y_extreme)
test_errors_extreme_ori = inv_yhat_extreme - inv_y_extreme
test_errors_extreme_ori.shape

(272, 12)

In [21]:
saved_model = load_model('../saved_models_mlp/load_all_weighted_META_95.h5')

yhat_extreme = saved_model.predict(test_X_extreme)
inv_yhat_extreme = scaler.inverse_transform(yhat_extreme)
inv_y_extreme = scaler.inverse_transform(test_y_extreme)
test_errors_extreme_re_meta = inv_yhat_extreme - inv_y_extreme

In [26]:
t_index = -1

# ========= Mann-Whitney U test =========
stat_mann, p_value_mann = mannwhitneyu(test_errors_extreme_ori[:,t_index], test_errors_extreme_re_meta[:,t_index], alternative='two-sided')
print(f"p_value_mann: {p_value_mann:.4e}")


# ========= wilcoxon U test =========
stat_wilcoxon, p_value_wilcoxon = wilcoxon(test_errors_extreme_ori[:, t_index], test_errors_extreme_re_meta[:, t_index])
print(f"p_value_wilcoxon: {p_value_wilcoxon:.4e}")


# ========= t-test =========
t_statistic, p_value = ttest_rel(test_errors_extreme_ori[:,t_index], test_errors_extreme_re_meta[:,t_index])
print(f"p_value_ttest: {p_value:.4e}")

p_value_mann: 2.0448e-03
p_value_wilcoxon: 3.5606e-07
p_value_ttest: 2.9265e-06
